In [43]:
from bs4 import BeautifulSoup
import random
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy import Table, Column, MetaData, Integer, Computed

In [44]:
conn = psycopg2.connect(
    dbname='postgres',  
    user='postgres',        
    password='11111',     
    host='localhost'              
)
conn.autocommit = True

In [45]:
cursor = conn.cursor()

create_prod_categories= """CREATE TABLE product_categories (
    id SERIAL PRIMARY KEY,
    parent_id INT,
    name VARCHAR(255),
    level INT
);
"""
cursor.execute(create_prod_categories)
conn.commit()  

In [46]:
create_products = """
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    category_id INT REFERENCES product_categories(id),
    name VARCHAR(255) NOT NULL,
    description TEXT,
    price DECIMAL(10, 2),
    url VARCHAR(255),
    image_url VARCHAR(255),
    rating FLOAT CHECK(rating BETWEEN 0 AND 5),
    created_at TIMESTAMP,
    updated_at TIMESTAMP
);
"""
cursor.execute(create_products)
conn.commit()

In [47]:
create_warehouses = """
CREATE TABLE warehouses (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    location VARCHAR(255)
);

"""
cursor.execute(create_warehouses)
conn.commit()

In [48]:
create_product_stock = """
CREATE TABLE product_stock (
    product_id INT REFERENCES products(id),
    warehouse_id INT REFERENCES warehouses(id),
    quantity INT,
    PRIMARY KEY (product_id, warehouse_id)
);
"""
cursor.execute(create_product_stock)
conn.commit()

In [49]:
create_users = """
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cursor.execute(create_users)
conn.commit()

In [50]:
create_sales = """
CREATE TABLE sales (
    id SERIAL PRIMARY KEY,
    user_id INT REFERENCES users(id),
    product_id INT REFERENCES products(id),
    quantity INT,
    price DECIMAL(10, 2)
);
"""
cursor.execute(create_sales)
conn.commit()

In [61]:
create_user_preferences = """
CREATE TABLE user_preferences (
    user_id INT PRIMARY KEY REFERENCES users(id),
    gender VARCHAR(10),
    age_range VARCHAR(10),
    categories TEXT
);
"""
cursor.execute(create_user_preferences)
conn.commit()

In [52]:
create_recommendations = """
CREATE TABLE recommendations (
    user_id INT REFERENCES users(id),
    product_id INT REFERENCES products(id),
    score FLOAT,
    PRIMARY KEY (user_id, product_id)
);
"""
cursor.execute(create_recommendations)
conn.commit()

In [53]:
df = pd.DataFrame(columns=['Name', 'Price', 'Link', 'Stars', 'Image'])

# Пробегаемся по страничкам товаров
for page in range(1,25):
    # Отправляем запрос на страничку каталога с помощью options
    url = f'https://www.dns-shop.ru/catalog/markdown/?p={page}'
    response = requests.options(url)

    # Парсим HTML-код с помощью BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Поиск всех элементов catalog-products
    elements = soup.find_all(class_='catalog-products')
    for element in elements:
        # Имя товара
        name = element.find(class_='catalog-product__name').text
        # Ссылка на товар
        link = element.find('a').get('href')
        # Оценка товара (0 если нет)
        try:
            stars = int(element.find(class_='catalog-product__rating').get('data-rating'))
        except ValueError:
            stars = 0
        # Ссылка на картинку товара
        image = element.find(class_='catalog-product__image-link').get('href')
        # Цена (ВРЕМЕННОЕ РЕШЕНИЕ)
        price = random.randint(1000, 15000)

        # Вывод результатов скрейпинга (дебаггинг)
        # ПРИ ПОЛУЧЕНИИ ВОЗМОЖНОСТИ СТОИТ ПОЛУЧИТЬ РЕАЛЬНЫЕ ЦЕНЫ
        # print(name)
        # print(price)
        # print(link)
        # print(stars)
        # print(image)

        # Вставка полученных данных в датафрейм
        df.loc[len(df)] = [name, price, link, stars, image]

# Вывод полученного датафрейма (дебаггинг)
# print(df)

# Сохранение результатов в excel
current_date = datetime.now().strftime('%Y-%m-%d')
filename = f'dns_parsed_data_{current_date}.xlsx'
# df.to_excel(filename, index=False, engine='openpyxl')
print(f'DataFrame saved to {filename}')

DataFrame saved to dns_parsed_data_2024-12-12.xlsx


In [54]:
products_df = df[['Name', 'Price', 'Link', 'Stars', 'Image']]
products_df['category_id'] = ''  # Оставляем пустым категорию
products_df['description'] = ''  # Оставляем пустым описание
products_df['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
products_df['updated_at'] = products_df['created_at']

In [55]:
product_categories_df = pd.DataFrame({
    'id': range(1, len(df) + 1),
    'parent_id': [None] * len(df),
    'name': df['Name'],
    'level': ''
})

In [56]:
product_stock_df = pd.DataFrame({
    'product_id': range(1, len(df) + 1),
    'warehouse_id': [random.randint(1, 5) for _ in range(len(df))],
    'quantity': [random.randint(1, 50) for _ in range(len(df))]
})

In [57]:
def get_short_name(full_name):
    words = full_name.split()
    short_name = ' '.join(words[:2])
    return short_name

products_df['Short_name'] = products_df['Name'].apply(lambda x: get_short_name(x))
products_df

,Name,Price,Link,Stars,Image,category_id,description,created_at,updated_at,Short_name
0,"FM-трансмиттер Ritmix FMT-A707[дисплей, Blueto...",6053,/catalog/markdown/f08ef9bb-98b6-11ef-8f17-0050...,0,/catalog/markdown/f08ef9bb-98b6-11ef-8f17-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,FM-трансмиттер Ritmix
1,"GPS навигатор NAVITEL E707 Magnetic[7"", 800x48...",13188,/catalog/markdown/0bcdcd27-52c7-11ef-b88f-0015...,4,/catalog/markdown/0bcdcd27-52c7-11ef-b88f-0015...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,GPS навигатор
2,"IP-камера Zodikam 3155-W[2592x1944, 25 кадр./с...",5911,/catalog/markdown/38a64ae0-a0da-11ef-8f1e-0050...,0,/catalog/markdown/38a64ae0-a0da-11ef-8f1e-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,IP-камера Zodikam
3,"KVM переключатель ATEN CS1732B[2 port, VGA (D-...",8893,/catalog/markdown/8e240b63-5847-11ef-b894-0015...,0,/catalog/markdown/8e240b63-5847-11ef-b894-0015...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,KVM переключатель
4,500 ГБ M.2 NVMe накопитель MSI SPATIUM M450 [S...,7079,/catalog/markdown/b3ca4642-b2ce-11ef-8f2f-0050...,0,/catalog/markdown/b3ca4642-b2ce-11ef-8f2f-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,500 ГБ
...,...,...,...,...,...,...,...,...,...,...
91,Ковш Polaris ECO collection-16SP 1.5 л фиолето...,10543,/catalog/markdown/0ca737d4-98ea-11ef-8f20-0050...,5,/catalog/markdown/0ca737d4-98ea-11ef-8f20-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,Ковш Polaris
92,Шкаф коммутационный 5Bites TC5402-06B[защита -...,11331,/catalog/markdown/09050b03-6e5f-11ef-8ef0-0050...,0,/catalog/markdown/09050b03-6e5f-11ef-8ef0-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,Шкаф коммутационный
93,Компактный фотоаппарат Rekam Presto Zoom 800 ч...,3599,/catalog/markdown/2cd5a439-ccaa-4097-b418-6729...,0,/catalog/markdown/2cd5a439-ccaa-4097-b418-6729...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,Компактный фотоаппарат
94,Компрессор поршневой масляный FinePower TRO260...,10164,/catalog/markdown/aef6cbd8-7235-11ef-8ef4-0050...,0,/catalog/markdown/aef6cbd8-7235-11ef-8ef4-0050...,,,2024-12-12 06:07:37,2024-12-12 06:07:37,Компрессор поршневой


In [60]:
products = [
    'fm-трансмиттер ritmix', 'gps навигатор', 'ip-камера zodikam',
    'kvm переключатель', '500 гб', '1024 гб', 'wi-fi роутер',
    'роутер huawei', 'автопроигрыватель prology',
    'автосигнализация starline', 'внутренний адаптер',
    'аккумуляторная батарея', 'аккумуляторная цепная',
    'дополнительный аккумулятор', 'аккумулятор einhell',
    'блок питания', 'умная колонка', 'аэрогриль dexp',
    'беззеркальный фотоаппарат', 'воздуходувка champion',
    'бензопила carver', 'бензопила patriot', 'бетоносмеситель вихрь',
    'блендер погружной', 'блендер стационарный', 'вакуумный упаковщик',
    'газовая варочная', 'индукционная варочная', 'вентилятор aceline',
    'вентилятор polaris', 'вентилятор aerocool',
    'пылесос вертикальный', 'моющий пылесос',
    'переходник однонаправленный', 'видеокарта msi',
    'видеорегистратор mio', 'видеорегистратор с', 'винный шкаф',
    'водонагреватель газовый', 'водонагреватель электрический',
    'встраиваемая микроволновая', 'встраиваемая посудомоечная',
    'встраиваемый холодильник', 'вытяжка полновстраиваемая',
    'вытяжка островная', 'тепловая пушка',
    'газонокосилка аккумуляторная', 'газонокосилка бензиновая',
    'гайковерт finepower', 'гладильная доска', 'гравировальная машина',
    'гриль dexp', 'декоративный светильник', 'держатель автомобильный',
    'детские часы', 'диктофон ritmix', 'диспенсер apexcool',
    'диспенсер mijia', 'доска магнитно-маркерная', 'насос дренажный',
    'электрический духовой', '12 тб', 'зарядное устройство',
    'точильный станок', 'игровой держатель', 'руль dexp', 'игра the',
    'ибп dexp', 'йогуртница kitfort', 'картридж лазерный',
    'карта памяти', 'квадрокоптер autel', 'клавиатура проводная',
    'ковш polaris', 'шкаф коммутационный', 'компактный фотоаппарат',
    'компрессор поршневой', 'стол компьютерный'
]

keywords = [
    "вытяжка", "водонагреватель", "газонокосилка", "пылесос", 
    "видеорегистратор", "блендер", "вентилятор", "аккумулятор", 
    "компрессор", "микроволновая", "холодильник", "гладильная",
    "диспенсер", "игровой", "декоративный", "гравировальная",
    "бензопила", "аэрогриль", "душ", "доска","шкаф","квадрокоптер",
    "руль","игра","диктофон","йогуртница","светильник","переходник",
    "адаптер", "роутер","навигатор","часы"
]

categories = {}
category_id = 1

# Функция для определения категории на основе ключевых слов
def get_category(product_name):
    for keyword in keywords:
        if keyword in product_name.lower():
            return keyword
    return None

# Определяем категории для каждого товара и заполняем словарь categories
for product in products:
    category = get_category(product)
    
    # Если категория найдена, добавляем её в словарь с уникальным id
    if category:
        if category not in categories:
            categories[category] = category_id
            category_id += 1

# Создаем DataFrame из списка продуктов
# Добавляем новый столбец category_id, заполняя его значениями из словаря categories
products_df['category_id'] = products_df['Name'].apply(
    lambda x: categories.get(get_category(x), 0)
)

# Выводим результат
print("Найденные категории и их идентификаторы:")
print(categories)

print("\nDataFrame с продуктами и категориями:")
display(products_df)


Найденные категории и их идентификаторы:
{'навигатор': 1, 'роутер': 2, 'адаптер': 3, 'аккумулятор': 4, 'аэрогриль': 5, 'бензопила': 6, 'блендер': 7, 'вентилятор': 8, 'пылесос': 9, 'переходник': 10, 'видеорегистратор': 11, 'шкаф': 12, 'водонагреватель': 13, 'микроволновая': 14, 'холодильник': 15, 'вытяжка': 16, 'газонокосилка': 17, 'гладильная': 18, 'гравировальная': 19, 'декоративный': 20, 'часы': 21, 'диктофон': 22, 'диспенсер': 23, 'доска': 24, 'игровой': 25, 'руль': 26, 'игра': 27, 'йогуртница': 28, 'квадрокоптер': 29, 'компрессор': 30}

DataFrame с продуктами и категориями:


,Name,Price,Link,Stars,Image,category_id,description,created_at,updated_at,Short_name
0,"FM-трансмиттер Ritmix FMT-A707[дисплей, Blueto...",6053,/catalog/markdown/f08ef9bb-98b6-11ef-8f17-0050...,0,/catalog/markdown/f08ef9bb-98b6-11ef-8f17-0050...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,FM-трансмиттер Ritmix
1,"GPS навигатор NAVITEL E707 Magnetic[7"", 800x48...",13188,/catalog/markdown/0bcdcd27-52c7-11ef-b88f-0015...,4,/catalog/markdown/0bcdcd27-52c7-11ef-b88f-0015...,1,,2024-12-12 06:07:37,2024-12-12 06:07:37,GPS навигатор
2,"IP-камера Zodikam 3155-W[2592x1944, 25 кадр./с...",5911,/catalog/markdown/38a64ae0-a0da-11ef-8f1e-0050...,0,/catalog/markdown/38a64ae0-a0da-11ef-8f1e-0050...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,IP-камера Zodikam
3,"KVM переключатель ATEN CS1732B[2 port, VGA (D-...",8893,/catalog/markdown/8e240b63-5847-11ef-b894-0015...,0,/catalog/markdown/8e240b63-5847-11ef-b894-0015...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,KVM переключатель
4,500 ГБ M.2 NVMe накопитель MSI SPATIUM M450 [S...,7079,/catalog/markdown/b3ca4642-b2ce-11ef-8f2f-0050...,0,/catalog/markdown/b3ca4642-b2ce-11ef-8f2f-0050...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,500 ГБ
...,...,...,...,...,...,...,...,...,...,...
91,Ковш Polaris ECO collection-16SP 1.5 л фиолето...,10543,/catalog/markdown/0ca737d4-98ea-11ef-8f20-0050...,5,/catalog/markdown/0ca737d4-98ea-11ef-8f20-0050...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,Ковш Polaris
92,Шкаф коммутационный 5Bites TC5402-06B[защита -...,11331,/catalog/markdown/09050b03-6e5f-11ef-8ef0-0050...,0,/catalog/markdown/09050b03-6e5f-11ef-8ef0-0050...,12,,2024-12-12 06:07:37,2024-12-12 06:07:37,Шкаф коммутационный
93,Компактный фотоаппарат Rekam Presto Zoom 800 ч...,3599,/catalog/markdown/2cd5a439-ccaa-4097-b418-6729...,0,/catalog/markdown/2cd5a439-ccaa-4097-b418-6729...,0,,2024-12-12 06:07:37,2024-12-12 06:07:37,Компактный фотоаппарат
94,Компрессор поршневой масляный FinePower TRO260...,10164,/catalog/markdown/aef6cbd8-7235-11ef-8ef4-0050...,0,/catalog/markdown/aef6cbd8-7235-11ef-8ef4-0050...,30,,2024-12-12 06:07:37,2024-12-12 06:07:37,Компрессор поршневой
